In [1]:
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import os
import numpy
from sklearn.preprocessing import LabelEncoder

In [2]:

X_fileName = []
y_val = []

In [3]:

for fileName in (glob.iglob('train/**/**', recursive=False)):
    X_fileName.append(fileName)
    y_val.append(fileName.split("\\")[-2])        

In [4]:
myDf = pd.DataFrame()
myDf['fileName'] = X_fileName
myDf['label'] = y_val

In [5]:

myLabelEncoder = LabelEncoder()
myDf['label_val'] = myLabelEncoder.fit_transform(myDf['label'])

In [6]:
def pointremove(image):
    
    img=image.copy()
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    for i in range(1, nb_components):
        if sizes[i]<100 :
            img[output==i]=0
    return img

In [7]:
def maxmeanpreprop(path):
    img1 = cv2.imread(path)
    g_img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img = g_img.copy()
    ret1,th1 = cv2.threshold(g_img,40,255,cv2.THRESH_BINARY)
    th1=pointremove(th1)
    blur = cv2.GaussianBlur(th1,(5,5),0)
    ret2,th2 = cv2.threshold(blur,130,255,cv2.THRESH_OTSU)

    a_img=th2&img
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(a_img, connectivity=4)
    sizes = stats[:, -1]

    max_label = 0
    max_mean = 0

    for i in range(1, nb_components):
        mean=np.mean(a_img[output==i])
        if mean > max_mean and sizes[i]>1000 :
            max_label = i
            max_mean = mean

    img3 = np.zeros(output.shape)
    img3[output == max_label] = 255
    new_image = img3[stats[max_label][1]:stats[max_label][1]+stats[max_label][3],stats[max_label][0]:stats[max_label][0]+stats[max_label][2]]
    image_for_input = cv2.resize(new_image, (100, 100), interpolation = cv2.INTER_NEAREST)
    return image_for_input

In [8]:

def my_generator(features, batch_size=16):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, 100, 100, 1))
    batch_labels = np.zeros((batch_size))
    while True:
        for i in range(batch_size):
            # choose random index in features
            index = np.random.choice(len(features),1)
            batch_features[i] = maxmeanpreprop(features.loc[index[0]]['fileName']).reshape([100, 100, 1])
            batch_labels[i] = features.loc[index[0]]['label_val']
        yield batch_features, batch_labels

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K

Using TensorFlow backend.


In [ ]:

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D

import tensorflow.compat.v1 as tf

run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)


# create model
model = Sequential()
#add model layers

model.add(Conv2D(100, kernel_size=3, activation="relu", input_shape=(100,100,1)))
model.add(Conv2D(100, kernel_size=3, activation="relu"))
model.add(Conv2D(100, kernel_size=3, activation="relu"))
model.add(Flatten())
model.add(Dense(16, activation="softmax"))

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# from keras.utils import to_categorical
# y_binary = to_categorical(np.array(Y))
# y_binary_test = to_categorical(np.array(Y_test))

# print((np.expand_dims(np.array(X), axis=-1))

# # Fit the model
model.fit_generator(my_generator(myDf), samples_per_epoch=200, nb_epoch=5)

C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=200, epochs=5)`


Epoch 1/5
200/200 [==============================] - 338s 2s/step - loss: 19.8981 - accuracy: 0.4659
Epoch 2/5
153/200 [=====================>........] - ETA: 1:20 - loss: 1.1508 - accuracy: 0.6773